## Facebook SQL interview Question

In [ ]:
https://datalemur.com/questions/updated-status

Learnings

You can use the UNION operator to join two columns with the same data types together.

Amazon.com can have a table of customers user_id and their average order value
and another table with customers user_id and a columns stating if they have ever had a prime membership

Amazon wants to know how many paying customers they have (customer swho purchased any product or service)

Not every customer who purchses has a prime membership.

We can use get all the customer_id's from the first table and all the customer_id's from the second table.
Then use the union operator to combine all the customer ids from both tables

In [ ]:
-- My solution
/* Took approximately 45 mins to solve because I didn't know how to use FULL OUTER JOIN 
to get a complete list of companies that advertised on Facebook.

Some companies didn't appear in the advertisers table so INNER JOINS would not give a full
list of all companies that advertised on Facebook
*/

-- Get all users all in one column
WITH all_users AS (
  SELECT user_id
  FROM advertiser

  UNION

  SELECT user_id
  FROM daily_pay
),

-- Join all_users, advertiser and daily paid table
ad_dpay2 AS (
  SELECT 
        au.user_id AS main_id,
        a.user_id,
        dp.user_id,
        a.status,
        dp.paid
        
  FROM all_users AS au 
  
  LEFT JOIN advertiser AS a 
  ON au.user_id = a.user_id

  LEFT JOIN daily_pay AS dp 
  ON au.user_id = dp.user_id

),

-- Create new column to get new status of advertisers
user_new_status AS (
  SELECT 
      main_id,
      status,
      paid,
      CASE
        WHEN status IN ('NEW', 'EXISTING', 'RESURRECT', 'CHURN') AND paid IS NULL THEN 'CHURN'
        WHEN status IN ('NEW', 'EXISTING', 'RESURRECT') AND paid >= 0 THEN 'EXISTING'
        WHEN status IN ('CHURN') AND paid >0 THEN 'RESURRECT'
        ELSE 'NEW'
      END AS new_status
  FROM ad_dpay2
)


-- Display new_status of users
SELECT main_id, new_status
FROM user_new_status
ORDER BY main_id;

In [ ]:
  -- Example iof UNION operator
  SELECT user_id
  FROM advertiser

  UNION

  SELECT user_id
  FROM daily_pay

### More Efficient solution


In [ ]:
-- Step 1: Merge tables using FULL OUTER JOIN
SELECT *
FROM advertiser
FULL OUTER JOIN daily_pay
ON advertiser.user_id = daily_pay.user_id;

-- Step 2: Assign payment status based on payment
SELECT   
  CASE 
    WHEN paid IS NULL THEN 'CHURN' 
    WHEN paid IS NOT NULL AND advertiser.status IN ('NEW','EXISTING','RESURRECT') THEN 'EXISTING'
    WHEN paid IS NOT NULL AND advertiser.status = 'CHURN' THEN 'RESURRECT'
    WHEN paid IS NOT NULL AND advertiser.status IS NULL THEN 'NEW'
  END AS new_status
FROM advertiser
FULL OUTER JOIN daily_pay
  ON advertiser.user_id = daily_pay.user_id;

-- Step 3: Select the user IDs with adjusted status
  SELECT 
  COALESCE(advertiser.user_id, daily_pay.user_id) AS user_id,
  CASE 
    WHEN paid IS NULL THEN 'CHURN' 
    WHEN paid IS NOT NULL AND advertiser.status IN ('NEW','EXISTING','RESURRECT') THEN 'EXISTING'
    WHEN paid IS NOT NULL AND advertiser.status = 'CHURN' THEN 'RESURRECT'
    WHEN paid IS NOT NULL AND advertiser.status IS NULL THEN 'NEW'
  END AS new_status
FROM advertiser
FULL OUTER JOIN daily_pay
  ON advertiser.user_id = daily_pay.user_id
ORDER BY user_id;

#### Errors I ran into and solved

If you are joining multiple tables and each table has columns that have the same name and wrapping the join in a CTE, it is advisable to give the namesake column you wish to primarily use an alias so you don't run in error later when you call the namesake column in another query.



# Amazon SQL Interview Question

[Amazon SQL Interview Question](https://datalemur.com/questions/sql-highest-grossing)

In [ ]:
-- My solution took about 1 hr to solve

-- Get the total amount cutomers spent on each product
WITH total_spent AS (
  SELECT
        category,
        product,
        user_id,
        spend,
        SUM(spend) OVER(PARTITION BY product) AS total_product_spend
  FROM product_spend
  WHERE EXTRACT(YEAR FROM transaction_date)  = 2022
),

-- Rank product sales from highest to lowest
top2 AS (

  SELECT
        *,
        DENSE_RANK() OVER(PARTITION BY category ORDER BY total_product_spend DESC) as top_products
  FROM total_spent
)

-- Get top 2 best selling products within each category
SELECT DISTINCT product, category,total_product_spend
FROM top2
WHERE top_products IN (1,2)
ORDER BY category, total_product_spend DESC;

In [ ]:
-- Efficient solution

-- Rank products sales from highest to lowest
WITH total_spent AS (

  SELECT
        category,
        product,
        SUM(spend) AS total_spend,
        DENSE_RANK() OVER(PARTITION BY category ORDER BY SUM(spend) DESC) AS top_products
  FROM product_spend
  WHERE EXTRACT(YEAR FROM transaction_date) = 2022
  GROUP BY category, product
)

-- Get top 2 best selling products within each category

SELECT category, product, total_spend
FROM total_spent
WHERE top_products IN (1,2)
ORDER BY category, total_spend DESC;

### Why did it take an hour to solve?

I didn't filter for trasactions for the year of 2022 because I didn't pay attention to the questiom

I could have used the GROUP BY keyword to group by category and product, then used the SUM aggregate function to calculate the total amount customers spent on a particular product in each category

By using the GROUP BY keyword I could ranked the top two products customers spent the most on in each category within the first CTE which would have avoided having a the 2nd CTE.

Also I wasn't aware that I can use aggregate funtions with ORDER BY even within the OVER CLAUSE
 Eaxample
 RANK() OVER (
      PARTITION BY category 
      ORDER BY SUM(spend) DESC) AS ranking

### Key learnings

I can only use the DISTINCT keyword right after the SELECT keyword

I can only use the DISTINCT keyword with one column

DENSE_RANK gives the same number/rank to ties but doesn't skip numbers for ties

RANK gives the same rank to ties but skips a numbers after a tie

The WHERE clause comes bofore the GROUP BY CLAUSE. Data is filtered with WHERE before it is GROUPED using GROUP BY

# FACEBOOK SQL QUESTION

https://datalemur.com/questions/user-retention

In [ ]:
-- My solution

-- Identify users who are active in the month of June
with cte AS (
  SELECT
        user_id,
        event_id,
        event_type,
        event_date,
        
        --Identify who was active the previous month
        LAG(EXTRACT (MONTH FROM event_date)) 
          OVER(PARTITION BY user_id ORDER BY event_date) AS prev_month,

        -- Extract month from date as a number so we use group by
        EXTRACT(MONTH FROM event_date) AS event_month
  FROM user_actions
)

-- Filter for users active in JULY and disply thr number of active users in July
SELECT event_month AS month, COUNT(DISTINCT user_id) AS monthly_actice_users
FROM cte
WHERE TO_CHAR(event_date, 'YYYY-MM') = '2022-07' AND prev_month = 6
GROUP BY event_month

In [ ]:
-- SOlution from Datalemur

SELECT 
  EXTRACT(MONTH FROM curr_month.event_date) AS mth, 
  COUNT(DISTINCT curr_month.user_id) AS monthly_active_users 
FROM user_actions AS curr_month

WHERE EXISTS (
  SELECT last_month.user_id 
  FROM user_actions AS last_month
  WHERE last_month.user_id = curr_month.user_id
    AND EXTRACT(MONTH FROM last_month.event_date) =
    EXTRACT(MONTH FROM curr_month.event_date - interval '1 month')
)

  AND EXTRACT(MONTH FROM curr_month.event_date) = 7

  AND EXTRACT(YEAR FROM curr_month.event_date) = 2022

GROUP BY EXTRACT(MONTH FROM curr_month.event_date);

My solution is more efficient because WHERE EXISTS solution performs a correlated subquery for each row in curr_month making it computationally more expensive.

WHERE EXISTS method cheks each user's history activity and checks if the date of the activity is in June.
This can be computationally expensive with O(n)^2 time complexity

CTE takes a modular approach and makes code easier to decode and. The LAG window function passes through the whole dataset once and gets the previous date of a user's activity.

Then we can simply filter for records in July and users that were active the previous month in June. This is much less computationally expensive

In [ ]:
-- EXTRACT FORMATS OF DATES FROM DATES

-- Example
TO_CHAR(event_date, 'YYYY-MM') = '2022-07'